In [4]:
import pandas as pd
import numpy as np
from numba import njit
import vectorbtpro as vbt

In [5]:
data = vbt.BinanceData.fetch(["BTCUSDT", "ETHUSDT"])

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
sub_data = data.loc["2021-02-18":"2021-02-24"]
vbt.settings.set_theme("dark")
sub_data.plot(symbol="BTCUSDT")

FigureWidget({
    'data': [{'close': array([51552.6 , 55906.  , 55841.19, 57408.57, 54087.67, 48891.  , 49676…

In [7]:
from collections import namedtuple

Signals = namedtuple("Signals", [  
     "entries", 
     "exits"
 ])
Orders = namedtuple("Orders", [  
     "size", 
     "accumulate"
 ])
Ladders = namedtuple("Ladders", [  
     "tp1", 
     "tp2", 
     "tp1_pct", 
     "tp2_pct"
 ])

ladder_info_dt = np.dtype([  
     ("init_idx", np.int_),
     ("init_price", np.float_),
     ("tp1", np.float_),
     ("tp2", np.float_),
     ("tp1_pct", np.float_),
     ("tp2_pct", np.float_)
 ], align=True)

In [8]:
Orders

__main__.Orders

In [9]:
Orders.size

_tuplegetter(0, 'Alias for field number 0')

In [10]:
type(Orders)

type

In [11]:
x = Orders(2,4)

In [12]:
type(x)

__main__.Orders

In [13]:
@njit(debug = True)
def signal_func_nb(c, signals, orders, ladders, last_ladder_info):  
     is_entry = vbt.pf_nb.select_nb(c, signals.entries)  
     is_exit = vbt.pf_nb.select_nb(c, signals.exits)
     position_now = c.last_position[c.col]
     position_type = "BUY" if position_now > 0 else ( "NoSignal" if position_now == 0  else "Sell")
     print(f"Entry: {str(is_entry).upper()} | Exit: {str(is_exit).upper()}")
     print(f"C.Col : {c.col} | Entry: {str(is_entry).upper()} | Exit: {str(is_exit).upper()} | Position: {position_type}")

    #  if position_now == 0 and is_entry:
        #  orders.size[c.col] = np.inf  
        #  orders.accumulate[c.col] = False
        #  return True, False, False, False
     # if position_now > 0 and is_exit:
     #     orders.size[c.col] = np.inf
     #     orders.accumulate[c.col] = False
     #     return False, True, False, False
     # if position_now == 0:
     #     return False, False, False, False

    #  print(f"C.Col : {c.col} | Entry: {str(is_entry).upper()} | Exit: {str(is_exit).upper()} | Position: {position_type} | Order.Size: {o_size}  | Order.Accumulate: {o_acc}")
     
    #  tp_info = c.last_tp_info[c.col]
    #  ladder_info = last_ladder_info[c.col]  
    #  last_order = c.order_records[c.order_counts[c.col] - 1, c.col]  
    #  if not vbt.pf_nb.is_stop_active_nb(tp_info):  
    #      if last_order.stop_type == -1:  
    #          ladder_info.init_idx = tp_info.init_idx  
    #          ladder_info.init_price = tp_info.init_price
    #          ladder_info.tp1 = vbt.pf_nb.select_nb(c, ladders.tp1)
    #          ladder_info.tp2 = vbt.pf_nb.select_nb(c, ladders.tp2)
    #          ladder_info.tp1_pct = vbt.pf_nb.select_nb(c, ladders.tp1_pct)
    #          ladder_info.tp2_pct = vbt.pf_nb.select_nb(c, ladders.tp2_pct)
    #          tp_info.stop = ladder_info.tp1  
    #          tp_info.exit_type = vbt.pf_enums.StopExitType.CloseReduce
    #          orders.size[c.col] = ladder_info.tp1_pct * position_now
    #          orders.accumulate[c.col] = True
    #      else:  
    #          vbt.pf_nb.set_tp_info_nb(  
    #              tp_info, 
    #              ladder_info.init_idx,  
    #              init_price=ladder_info.init_price,
    #              stop=ladder_info.tp2,
    #              exit_type=vbt.pf_enums.StopExitType.CloseReduce
    #          )
    #          orders.size[c.col] = ladder_info.tp2_pct * position_now  
    #          orders.accumulate[c.col] = True
     return False, False, False, False

In [14]:
pf = vbt.Portfolio.from_signals(
     sub_data,
     signal_func_nb = signal_func_nb,
     signal_args=(  
         vbt.RepEval(
             "Signals(entries, exits)",
             context=dict(Signals=Signals)  
         ),
         vbt.RepEval(
             "Orders(size, accumulate)",
             context=dict(Orders=Orders)
         ),
         vbt.RepEval(
             "Ladders(tp1, tp2, tp1_pct, tp2_pct)",
             context=dict(Ladders=Ladders)
         ),
         vbt.RepEval(
             "np.empty(wrapper.shape_2d[1], dtype=ladder_info_dt)", 
             context=dict(ladder_info_dt=ladder_info_dt)
         )
     ),
     size=vbt.RepEval("np.full(wrapper.shape_2d[1], np.inf)"),
     accumulate=vbt.RepEval("np.full(wrapper.shape_2d[1], False)"),
     broadcast_named_args=dict(
         entries=pd.Series([bool(1.), bool(0.),bool(0.), bool(0.), bool(0.), bool(0.), bool(0.)]), 
         exits=  pd.Series([bool(0.), bool(0.), bool(0.), bool(0.), bool(0.), bool(0.), bool(0.)]),
         tp1=0.05,
         tp2=0.1,
         tp1_pct=0.5,
         tp2_pct=1.0
     )
 )


/opt/miniconda3/envs/vbt/lib/python3.10/site-packages/vectorbtpro/portfolio/nb/from_signals.py:2678: NumbaDebugInfoWarning:

Could not find source for function: <function signal_func_nb at 0x167859510>. Debug line information may be inaccurate.



Entry: TRUE | Exit: FALSE
C.Col : 0 | Entry: TRUE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 0 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: TRUE | Exit: FALSE
C.Col : 1 | Entry: TRUE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 1 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 1 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 1 | Entry: FALSE | Exit: FALSE | Position: NoSignal
Entry: FALSE | Exit: FALSE
C.Col : 1 | Entry: F

In [15]:
pf.asset_flow

symbol,BTCUSDT,ETHUSDT
Open time,,
2021-02-18 00:00:00+00:00,0.0,0.0
2021-02-19 00:00:00+00:00,0.0,0.0
2021-02-20 00:00:00+00:00,0.0,0.0
2021-02-21 00:00:00+00:00,0.0,0.0
2021-02-22 00:00:00+00:00,0.0,0.0
2021-02-23 00:00:00+00:00,0.0,0.0
2021-02-24 00:00:00+00:00,0.0,0.0
